In [17]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K

In [18]:
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
data = pd.read_csv("feature.csv")

In [5]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [6]:
data = data.drop(columns = "File")

In [7]:
encoder = LabelEncoder()
Y = encoder.fit_transform(y)

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [11]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpqa2vs_ed.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpqa2vs_ed.py, line 48)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
224/224 [==============================] - 11s 4ms/step - loss: 1.3620 - accuracy: 0.4093 - f1_m: 0.1779 - precision_m: 0.4893 - recall_m: 0.1168 - val_loss: 1.0085 - val_accuracy: 0.5808 - val_f1_m: 0.5627 - val_precision_m: 0.7996 - val_recall_m: 0.4350
Epoch 2/50
224/224 [==============================] - 0s 2ms/step - loss: 0.9794 - accuracy: 0.6054 - f1_m: 0.5776 - precision_m:

224/224 [==============================] - 0s 2ms/step - loss: 0.4703 - accuracy: 0.8142 - f1_m: 0.8119 - precision_m: 0.8659 - recall_m: 0.7645 - val_loss: 0.4994 - val_accuracy: 0.8104 - val_f1_m: 0.8093 - val_precision_m: 0.8685 - val_recall_m: 0.7580
Epoch 29/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4587 - accuracy: 0.8243 - f1_m: 0.8224 - precision_m: 0.8755 - recall_m: 0.7757 - val_loss: 0.5090 - val_accuracy: 0.7977 - val_f1_m: 0.7997 - val_precision_m: 0.8551 - val_recall_m: 0.7515
Epoch 30/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.8239 - f1_m: 0.8230 - precision_m: 0.8750 - recall_m: 0.7771 - val_loss: 0.4680 - val_accuracy: 0.8194 - val_f1_m: 0.8179 - val_precision_m: 0.8654 - val_recall_m: 0.7757
Epoch 31/50
224/224 [==============================] - 0s 2ms/step - loss: 0.4460 - accuracy: 0.8270 - f1_m: 0.8238 - precision_m: 0.8732 - recall_m: 0.7800 - val_loss: 0.4702 - val_accuracy: 0.8224 - val_f1_m: 

In [12]:
y_pred = model.predict(X_test)

In [13]:
predicted = np.argmax(y_pred, axis=1)

In [14]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8483826010415902


In [15]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.93      0.95      0.94      2735
  Defacement       0.76      0.75      0.76      2499
     Malware       0.82      0.93      0.87      2834
    Phishing       0.85      0.68      0.75      2477
        Spam       0.87      0.90      0.89      3088

    accuracy                           0.85     13633
   macro avg       0.85      0.84      0.84     13633
weighted avg       0.85      0.85      0.85     13633



In [16]:
os.chdir("../")
os.chdir("models")
model.save("Model_v1")

INFO:tensorflow:Assets written to: Model_v1/assets


In [19]:
os.chdir("../")
os.chdir("scripts")

In [22]:
import data_creation_v3 as d

In [ ]:
d.UrlFeaturizer('')